### Day 15: Warehouse Woes

https://adventofcode.com/2024/day/15

In [1]:
from pathlib import Path
import sys

sys.path.append("..")

from tools import get_input


def parse(text: str) -> tuple[list[list[str]], str]:
    area_str, moves_str = text.split("\n\n")
    moves = "".join(moves_str.split("\n"))
    area = [list(row) for row in area_str.split("\n")]
    return area, moves


tst1 = parse((Path().parent / "test1.txt").read_text(encoding="utf-8"))
tst = parse((Path().parent / "test.txt").read_text(encoding="utf-8"))
inp = parse(get_input(15))

In [2]:
dir = {"^": (-1, 0), "v": (1, 0), "<": (0, -1), ">": (0, 1)}


def pushable(wh, i, j, di, dj) -> bool:
    while wh[i][j] != "#":
        if wh[i][j] == ".":
            wh[i][j] = "O"
            return True
        i += di
        j += dj
    return False

def get_start(wh: list[list[str]]) -> tuple[int, int]:
    for i in range(len(wh)):
        for j in range(len(wh[i])):
            if wh[i][j] == "@":
                return i, j
    raise ValueError("No start found")


def solution_1(wh, moves) -> int:
    wh = [row.copy() for row in wh]
    pi, pj = get_start(wh)
    for m in moves:
        wh[pi][pj] = "."
        di, dj = dir[m]
        ni, nj = pi + di, pj + dj
        if wh[ni][nj] == "O":
            if pushable(wh, ni, nj, di, dj):
                wh[pi][pj] = "."
                pi, pj = ni, nj
        elif wh[ni][nj] == ".":
            pi, pj = ni, nj
        wh[pi][pj] = "@"
    return sum(
        (i * 100 + j)
        for i in range(len(wh))
        for j in range(len(wh[i]))
        if wh[i][j] == "O"
    )


assert solution_1(*tst1) == 2028
assert solution_1(*tst) == 10092
solution_1(*inp)  # 1413675

1413675

In [3]:
from IPython.display import clear_output
from time import sleep

def push2(wh, i, j, di, dj) -> None:
    if dj != 0:
        if wh[i][j+dj] in "[]":
            push2(wh, i, j+dj, di, dj)
        wh[i][j+dj] = wh[i][j]
        wh[i][j] = "."
    else:
        if wh[i+di][j] == "[":
            push2(wh, i+di, j, di, dj)
        if wh[i+di][j] == "]":
            push2(wh, i+di, j-1, di, dj)
        if wh[i+di][j+1] == "[":
            push2(wh, i+di, j+1, di, dj)
        wh[i+di][j] = wh[i][j]
        wh[i][j] = "."
        wh[i+di][j+1] = wh[i][j+1]
        wh[i][j+1] = "."
        
 
def pushable2(wh, i, j, di, dj) -> bool:
    if dj != 0:
        if wh[i][j+dj] in "[]":
            return pushable2(wh, i, j+dj, di, dj)
        return wh[i+di][j+dj] == "."
    else:
        if wh[i+di][j] == "#" or wh[i+di][j+1] == "#":
            return False
        if wh[i+di][j] == "[":
            return pushable2(wh, i+di, j, di, dj)
        if wh[i+di][j] == "]":
            if not pushable2(wh, i+di, j-1, di, dj):
                return False
            if wh[i+di][j+1] == "[":
                return pushable2(wh, i+di, j+1, di, dj)
            return True
        if wh[i+di][j+1] == "[":
            return pushable2(wh, i+di, j+1, di, dj)
        return wh[i+di][j+dj] == "." and wh[i+di][j+1] == "."


def solution_2(wo, moves, copy=False, interactive=0) -> int:
    if copy:
        wh = [row.copy() for row in wo]
    else:
        wh = []
        for i in range(len(wo)):
            row = []
            for j in range(len(wo[i])):
                if wo[i][j] == "#":
                    row.append("#")
                    row.append("#")
                elif wo[i][j] == "O":
                    row.append("[")
                    row.append("]")
                elif wo[i][j] == "@":
                    row.append("@")
                    row.append(".")
                else:
                    row.append(".")
                    row.append(".")
            wh.append(row)
    pi, pj = get_start(wh)

    if interactive > 0:
        for row in wh:
            print("".join(row))
   
    cnt = 0
    for m in moves:
        if interactive and cnt > interactive:
            print(cnt)
            sleep(1)
            clear_output(wait=True)
            print(m)
        cnt += 1
        di, dj = dir[m]
        ni, nj = pi + di, pj + dj
        if wh[ni][nj] in "[]":
            if pushable2(wh, ni, nj - (1 if wh[ni][nj] == "]" and di != 0 else 0), di, dj):
                push2(wh, ni, nj - (1 if wh[ni][nj] == "]" and di != 0 else 0), di, dj)
                wh[pi][pj] = "."
                pi, pj = ni, nj
        elif wh[ni][nj] == ".":
            wh[pi][pj] = "."
            pi, pj = ni, nj
        wh[pi][pj] = "@"
        if interactive > 0 and cnt > interactive:
            for row in wh:
                print("".join(row))
    if interactive > 0:
        for row in wh:
            print("".join(row))
    return sum(
        (i * 100 + j)
        for i in range(len(wh))
        for j in range(len(wh[i]))
        if wh[i][j] == "["
    )

In [4]:
tst2 = parse((Path().parent / "test2.txt").read_text(encoding="utf-8"))
assert solution_2(*tst2, interactive=1000) == 618

##############
##......##..##
##..........##
##....[][]@.##
##....[]....##
##..........##
##############
##############
##...[].##..##
##...@.[]...##
##....[]....##
##..........##
##..........##
##############


In [5]:
assert solution_2(*tst, interactive=10000) == 9021

####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################
####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################


In [6]:
solution_2(*inp)  # 1399772

1399772